# Exploring SQL with DuckDB

We can use DuckDB to run an in memory SQL database which can be used for a variety of use cases. Install it in Conda with:

```
conda install python-duckdb
```

Will will also want

```
conda install sqlalchemy ipython-sql duckdb-engine
```

In [1]:
from pathlib import Path

In [2]:
import duckdb
import pandas as pd
import sqlalchemy
import pymongo
from dotenv import dotenv_values

In [3]:
local_data_path = Path("../../../datafiles/")

## Demo 1: Filtering large mixed light curve datasets
I have some Swift space telescope light curve data in a cloud based implementation of Mongo DB. If you want to experiment there is a free-tier of [Mongo Atlas](https://www.mongodb.com/cloud/atlas/register) which you can register for if you want to experiment.

There is a local snapshot of the data in the top-level datafiles directory: `datafiles/swift-lc/lightcurve_data.json`. You can load the data directly from the file to follow along; as an extention exercise you could try creating your own Mongo Atlas DB and uploading the data there, or you could use Docker to run a local Mongo database and use that akin to what we did in the Day 1 demo.

### ASIDE: Securing credentials when sharing code
The challenge is that I have security and I need a secure way to share this code without exposing my username, password etc; `python-dotenv` library to the rescue. This little library allows you to pull environment variables out of files, to be secure I make them hidden files and make sure to add them to my `.gitignore` file.

So I locally have a `.env` file that has the following lines in it:
```
MONGO_USER = <username>
MONGO_PASS = <password>
MONGO_URI = <url_path_to_database>
MONGO_CONNECTION_STRING = mongodb+srv://${MONGO_USER}:${MONGO_PASS}@${MONGO_URI}/?retryWrites=true&w=majority
```

This is a really good way to share code without exposing sensitive data; you can also load the file in as actual environment variables which is very useful when doing local development on code that is actually deployed locally. Check out the [python-dotenv documentation](https://pypi.org/project/python-dotenv/) for more details.

In [4]:
config = dotenv_values(".env")

config.keys()

odict_keys(['MONGO_USER', 'MONGO_PASS', 'MONGO_URI', 'MONGO_CONNECTION_STRING'])

In [5]:
config.get("MONGO_USER")

'api-access'

## Connecting to Mongo

Now that we have loaded the connection details I can establish a connection to the Mongo server, connect to the "SwiftLC" database and access the collection of "lightcurves" records.

In [6]:
client = pymongo.MongoClient(config.get("MONGO_CONNECTION_STRING"))

db = client["SwiftLC"]

collection = db["lightcurves"]

If I return a single record we can see the form of the data

In [7]:
collection.find_one()

{'DATETIME': datetime.datetime(2005, 2, 13, 0, 0),
 'SOURCE': 'IGR J06074+2205',
 'DATA_FLAG': 0,
 'DAY': 44,
 'TIMEDEL_EXPO': 832.0,
 'TIMEDEL_DITHERED': 832.0,
 'RATE': 0.00155646,
 'TIMEDEL_CODED': 812.5,
 'ERROR': 0.0018947,
 'SYS_ERR': 5.448e-05,
 '_id': ObjectId('642880cb8dac64a4cfe06aa8'),
 'YEAR': 2005,
 'STAT_ERR': 0.0016215,
 'TIME': 53414}

We will build a local dataframe of a sample of 200,000 light curve datapoints from multiple sources.

In [ ]:
lightcurve_df = pd.DataFrame(collection.find({}, {"_id": 0}).limit(200_000))
lightcurve_df.head()

In [ ]:
# Uncomment this cell if you are loading the local copy of the data directly from file

#import json
#with open(local_data_path/"swift-lc"/"lightcurve_data.json", "r") as fileIn:
#    lightcurve_df = pd.DataFrame(json.load(fileIn))
#lightcurve_df.head()

In [ ]:
lightcurve_df.SOURCE.unique()[0:15]

As an initial test let's see how long it takes to find all datapoints that have an exposure time of less than 200 seconds

In [ ]:
exposure_time = 200

In [ ]:
%%timeit
test_result = lightcurve_df.query("TIMEDEL_EXPO < @exposure_time")

## Introducing DuckDB

By passing the dataframe into DuckDB - an in-memory SQL database can we improve our calculation time?

In [ ]:
dbcon = duckdb.connect()

In [ ]:
dbcon.execute("SELECT 1, 2, 3").fetchall()

In [ ]:
def dbrun(query: str) -> pd.DataFrame:
    """Run the query with duckdb"""
    result = dbcon.query(query).to_df()
    return result

In [ ]:
query = f"SELECT * FROM lightcurve_df WHERE TIMEDEL_EXPO < {exposure_time}"
exposure_under_200s_df = dbrun(query)

In [ ]:
%%timeit
exposure_under_200s_df

We can see that we get a massive speedup - over 100,000 times faster!

## Exercise 1: Using DuckDB to complete our word count exercise

As a reminder there are a few stages that we will have to follow to perform word counts across all of our books.

### 1. Process a book into tokens
We will first need a function that does the following:
1. Read the data from a book file.
2. Split all the words into individual tokens.
3. Strip out any non-alphanumeric characters.
4. Remove any "blank" tokens

Write a function to do this, we've helped you out by providing a function that strips the non-alpha characters. 

In [ ]:
def strip(s: str) -> str:
    """Strip removes any non-alpha charcters"""
    return ''.join(filter(str.isalpha, s))

In [ ]:
def read_book(file_path: Path):
    with open(file_path, "r") as fileIn:
        data = fileIn.read()
    tokens = data.split()
    stripped = map(strip, tokens)
    notempty = filter(lambda w: len(w)>0, stripped)
    return notempty

### 2. Looping over all of our book files anc creating a "master" list of words

We have provided you with the root-folder where the book data is stored.

Write a piece of code that finds all of the books (`.txt`) files in the given folder, extract all the individual tokens and put them into a master list of words called `words`.

In [ ]:
books_root = local_data_path/"books/"

In [ ]:
# write your code here ...

book_files = books_root.glob("*.txt")

words = []
for book in book_files:
    words += list(read_book(book))

We can now turn this into a dataframe and see how long it would take to count all the occurences of each unique word using the built in dataframe `.value_counts` function

In [ ]:
words_df = pd.DataFrame(words, columns=["word"])

print(f"There are {len(words_df)} words in all our books")

In [ ]:
%%timeit
res2 = words_df.value_counts

We now have a baseline for how long it could take to perform this task.

### 3. Leveraging DuckDB & SQL to count all the words in the dataframe

The final part of the exercise is to write the SQL query tht can count all of occurences of each unique word in the dataframe. Bonus points for sorting the words from most frequent to least frequent... 

In [ ]:
#write your query here ...
wordcount_query = f"SELECT word, COUNT(word) AS COUNT FROM words_df GROUP BY word ORDER BY COUNT DESC"

In [ ]:
wordcount_df = dbrun(wordcount_query)

In [ ]:
%%timeit
wordcount_df

In [ ]:
wordcount_df.head()

In [ ]:
wordcount_df.tail()

## Conclusion

We can see that we again get another speedup although this time it is only a factor of 3.